# Implementacja CNN VGG16 dla klasyfikacji emocji z danych audio

## Implementacja i analiza 

Ten notebook przedstawia implementację głębokiej sieci neuronowej bazującej na architekturze VGG16 do rozpoznawania emocji z nagrań głosowych. W projekcie wykorzystujemy reprezentację nagrań audio w formie mel-spektrogramów, które są następnie analizowane przez sieć konwolucyjną.

### Architektura VGG16

VGG16 (Visual Geometry Group 16) to głęboka sieć konwolucyjna składająca się z 16 warstw z wagami, opracowana przez Karen Simonyan i Andrew Zissermana z Uniwersytetu Oksfordzkiego. Wyróżnia się:

- **Strukturą sekwencyjną**: 13 warstw konwolucyjnych i 3 warstwy w pełni połączone
- **Prostym i jednorodnym designem**: Wykorzystanie małych filtrów konwolucyjnych (3x3) przez całą sieć
- **Głębokością**: Sekwencja warstw z rosnącą liczbą filtrów (64-512)
- **Zastosowaniem max-poolingu**: Redukcja wymiarowości przy zachowaniu istotnych cech

W tym projekcie adaptujemy architekturę VGG16 do rozpoznawania wzorców w spektrogramach audio reprezentujących różne emocje w głosie. Wykorzystujemy istniejącą implementację architektury VGG16 z biblioteki torchvision, z odpowiednimi modyfikacjami warstw, które dostosowują model do naszego zadania klasyfikacji emocji.

### Mel-spektrogramy jako reprezentacja danych audio

Zamiast bezpośredniej analizy surowego sygnału audio, przekształcamy nagrania do mel-spektrogramów, co pozwala na:

- **Wizualizację dźwięku**: Dwuwymiarowa reprezentacja, gdzie:
  - Oś pozioma reprezentuje czas
  - Oś pionowa reprezentuje częstotliwości w skali mel (lepiej odpowiadającej ludzkiemu słyszeniu)
  - Intensywność kolorów odzwierciedla energię w danym paśmie częstotliwości

- **Wydobycie cech charakterystycznych dla emocji**:
  - Wzorce intonacji i modulacji głosu
  - Charakterystyczne sygnatury częstotliwościowe dla różnych stanów emocjonalnych
  - Zmiany w dynamice i barwie głosu

Ta reprezentacja umożliwia wykorzystanie architektury CNN (standardowo używanej do analizy obrazów) do efektywnej analizy wzorców dźwiękowych odpowiadających różnym emocjom.

### Moduły pomocnicze

W implementacji wykorzystujemy następujące moduły pomocnicze:

##### `helpers/data.py`
- `load_nemo_dataset()`: Funkcja ładująca zbiór danych NeMo Emotion Dataset
- `prepare_data()`: Przetwarza dane audio do mel-spektrogramów i dzieli je na zbiory treningowy, walidacyjny i testowy
- `predict_emotion()`: Umożliwia klasyfikację emocji dla nowych nagrań audio

##### `helpers/VGG16_definition.py`
- `build_vgg16_model()`: Tworzy model VGG16 dostosowany do klasyfikacji audio
- `train_model()`: Implementuje proces trenowania modelu z zapisem historii
- `save_model()`: Zapisuje wytrenowany model do późniejszego wykorzystania

##### `helpers/utils.py`
- `evaluate_model()`: Generuje metryki oceny modelu oraz macierz pomyłek
- `plot_training_history()`: Wizualizuje przebieg treningu (dokładność i stratę)

##### `config.py`
Zawiera globalne parametry konfiguracyjne projektu:
- Parametry uczenia (LEARNING_RATE, BATCH_SIZE, EPOCHS)
- Liczba klas emocji (NUM_CLASSES)
- Parametry ekstrakcji cech audio

## 1. Ustawienia środowiska i importy

In [8]:
import sys
import os

# Dodaj katalog główny projektu do sys.path
current_dir = (
    os.path.dirname(os.path.abspath(__file__))
    if "__file__" in globals()
    else os.getcwd()
)
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Katalog główny projektu: {project_root}")
print(f"Czy katalog src istnieje: {os.path.exists(os.path.join(project_root, 'src'))}")

import warnings

warnings.filterwarnings("ignore")

# Importowanie bibliotek
from tqdm import tqdm

# Importowanie PyTorch
import torch
from torch.utils.data import TensorDataset, DataLoader


# Importowanie konfiguracji
from src.config import (
    VGG16_BATCH_SIZE as BATCH_SIZE,
    VGG16_EPOCHS as EPOCHS,
    VGG16_LEARNING_RATE as LEARNING_RATE,
    VGG16_NUM_CLASSES as NUM_CLASSES,
    VGG16_MODEL_DIR,
)

# Podmiana tqdm w module vgg16_data
import src.helpers.vgg16_data

src.helpers.vgg16_data.tqdm = tqdm

# Importowanie stworzonych modułów
from src.helpers.vgg16_data import load_nemo_dataset, prepare_data
from src.helpers.VGG16_definition import build_vgg16_model, train_model, save_model
from src.helpers.vgg16_utils import evaluate_model, plot_training_history

In [ ]:
# Sprawdzenie i konfiguracja dostępności GPU
print("Sprawdzanie dostępności GPU...")
if torch.cuda.is_available():
    device = torch.device("cuda")
    current_device = torch.cuda.current_device()
    print(
        f"✅ GPU zostało poprawnie skonfigurowane: {torch.cuda.get_device_name(current_device)}"
    )

    # Wyświetlenie informacji o pamięci GPU
    print(f"   Używana karta: {torch.cuda.get_device_name(0)}")
    print(
        f"   Całkowita pamięć: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB"
    )
    print(f"   Architektura: {torch.cuda.get_device_capability(0)}")


else:
    device = torch.device("cpu")
    print("❌ Brak dostępnego GPU z obsługą CUDA. Używanie CPU...")

## 2. Ładowanie i przygotowanie danych

In [ ]:
print("📂 Rozpoczynam ładowanie i przygotowanie danych...")
try:
    dataset = load_nemo_dataset()

    # Przygotowanie danych - usunięto for_torch=True, jeśli wywołuje błąd
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(dataset)

    # Konwersja danych do formatów PyTorch
    print("🔄 Konwersja danych do formatu PyTorch...")
    X_train_tensor = torch.FloatTensor(X_train).permute(
        0, 3, 1, 2
    )  # [batch, channels, height, width]
    X_val_tensor = torch.FloatTensor(X_val).permute(0, 3, 1, 2)
    X_test_tensor = torch.FloatTensor(X_test).permute(0, 3, 1, 2)

    y_train_tensor = torch.LongTensor(y_train)
    y_val_tensor = torch.LongTensor(y_val)
    y_test_tensor = torch.LongTensor(y_test)

    # Tworzenie DataLoaderów
    print("🔄 Tworzenie DataLoaderów...")
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
    )
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=2)

    # Podsumowanie danych
    print("✅ Dane zostały pomyślnie przygotowane:")
    print(
        f"   ▪ Zbiór treningowy: {X_train_tensor.shape[0]} próbek, kształt: {X_train_tensor.shape}"
    )
    print(
        f"   ▪ Zbiór walidacyjny: {X_val_tensor.shape[0]} próbek, kształt: {X_val_tensor.shape}"
    )
    print(
        f"   ▪ Zbiór testowy: {X_test_tensor.shape[0]} próbek, kształt: {X_test_tensor.shape}"
    )
    print(f"   ▪ Batch size: {BATCH_SIZE}")
except Exception as e:
    print(f"❌ Wystąpił błąd podczas przygotowywania danych: {str(e)}")
    import traceback

    traceback.print_exc()

## 3.  Budowa i trenowanie modelu

In [ ]:
try:
    print("🔄 Budowanie modelu VGG16...")
    model = build_vgg16_model(num_classes=NUM_CLASSES)
    model = model.to(device)
    print(model)

    # Trenowanie modelu
    print("🚀 Rozpoczynam trenowanie modelu...")
    model, history = train_model(
        model,
        train_loader,
        val_loader,
        device,
        epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
    )
    print("✅ Trenowanie zakończone!")
except Exception as e:
    print(f"❌ Wystąpił błąd podczas trenowania modelu: {str(e)}")
    import traceback

    traceback.print_exc()

## 4. Ewaluacja modelu

In [ ]:
try:
    print("🔍 Ewaluacja modelu na zbiorze testowym...")

    # Definiujemy folder zapisu - używamy zmiennej z config
    save_dir = VGG16_MODEL_DIR

    # Ewaluacja modelu z zapisem znormalizowanej macierzy pomyłek
    test_acc, test_loss, confusion_matrix = evaluate_model(
        model, test_loader, device, save_dir=save_dir, normalize=True
    )

    # Wizualizacja wyników historii treningu
    print("📊 Wizualizacja wyników trenowania...")
    plot_training_history(history, save_dir=save_dir)

    print(f"✅ Wykresy i macierz pomyłek zapisane w katalogu: {save_dir}")

except Exception as e:
    print(f"❌ Wystąpił błąd podczas ewaluacji modelu: {str(e)}")
    import traceback

    traceback.print_exc()

## 5. Zapisanie modelu

In [15]:
try:
    print("💾 Zapisywanie modelu...")

    # Użyj funkcji save_model z helpers, która automatycznie używa VGG16_MODEL_DIR
    save_model(model, history, "best_model_vgg16.pt")

    print("✅ Model został pomyślnie zapisany!")
except Exception as e:
    print(f"❌ Błąd podczas zapisywania: {str(e)}")

💾 Zapisywanie modelu...
✅ Model został pomyślnie zapisany!
